In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23661198
paper_name = 'vahey_voldman_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data1 = pd.read_excel('raw_data/c3lc50162k.xlsx', sheet_name='Sheet1', skiprows=1)
original_data2 = pd.read_excel('raw_data/c3lc50162k_2.xlsx', sheet_name='Sheet1', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 4469 x 8
Original data dimensions: 4485 x 8


In [7]:
original_data2.head()

,Unnamed: 0,Unnamed: 1,y (1),y (2),y (avg.),Δy,Average,Spread
0,YAL004W,YAL004W,0.3423,0.3109,0.3266,0.0222,0.2392,0.0417
1,YAL005C,SSA1,-0.3448,-0.2752,-0.3100,0.0492,0.2270,0.0433
2,YAL008W,FUN14,0.0119,0.0326,0.0223,0.0146,0.2329,0.0429
3,YAL010C,MDM10,-0.8221,-0.6043,-0.7132,0.1540,0.2178,0.0472
4,YAL011W,SWC1,0.0498,-0.195,-0.0726,0.1731,0.2304,0.0414


In [8]:
original_data1['orf'] = original_data1['Unnamed: 0'].astype(str)
original_data2['orf'] = original_data2['Unnamed: 0'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data1['orf'] = clean_orf(original_data1['orf'])
original_data2['orf'] = clean_orf(original_data2['orf'])

In [12]:
# Translate to ORFs 
original_data1['orf'] = translate_sc(original_data1['orf'], to='orf')
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orf'])
print(original_data1.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, y (1), y (2), y (avg.), Δy, Average, Spread, orf, orfs]
Index: []


In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, y (1), y (2), y (avg.), Δy, Average, Spread, orf, orfs]
Index: []


In [15]:
original_data1['data'] = original_data1['y (avg.)'].astype(float)
original_data2['data'] = original_data2['y (avg.)'].astype(float)

In [16]:
original_data1.set_index('orf', inplace=True)
original_data2.set_index('orf', inplace=True)

In [17]:
original_data1 = original_data1[['data']].copy()
original_data2 = original_data2[['data']].copy()

In [18]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [20]:
original_data = original_data1.join(original_data2, lsuffix='_1', rsuffix='_2', how='outer')

In [21]:
original_data.head()

,data_1,data_2
orf,,
YAL004W,0.1422,0.3266
YAL005C,0.6907,-0.3100
YAL008W,0.6061,0.0223
YAL010C,1.0975,-0.7132
YAL011W,0.6044,-0.0726


# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [95, 16597]

In [24]:
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,95,16597
data_type,value,value
orf,,
YAL004W,0.1422,0.3266
YAL005C,0.6907,-0.3100
YAL008W,0.6061,0.0223
YAL010C,1.0975,-0.7132
YAL011W,0.6044,-0.0726


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,95,16597
,data_type,value,value
gene_id,orf,,
1863,YAL004W,0.1422,0.3266
4,YAL005C,0.6907,-0.3100
6,YAL008W,0.6061,0.0223
8,YAL010C,1.0975,-0.7132
9,YAL011W,0.6044,-0.0726


In [29]:
data.shape

(4442, 2)

# Normalize

In [30]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [31]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [32]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,95,16597,95,16597
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1863,YAL004W,0.1422,0.3266,-0.129249,0.935492
4,YAL005C,0.6907,-0.3100,1.064417,-0.774845
6,YAL008W,0.6061,0.0223,0.880307,0.117937
8,YAL010C,1.0975,-0.7132,1.949709,-1.858112
9,YAL011W,0.6044,-0.0726,0.876608,-0.137029


# Print out

In [33]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
from IO.save_data_to_db3 import *

In [35]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 23661198...
Inserting the new data...


100%|██████████| 2/2 [00:14<00:00,  7.12s/it]

Updating the data_modified_on field...
